In [5]:
import os
import json
import argparse
import logging
import pickle
import random

import pandas as pd
import torch
from torch.utils import data
import lightning.pytorch as pl
from lightning.pytorch.loggers.csv_logs import CSVLogger

from proteinclip import data_utils, fasta_utils, swissprot, hparams
from proteinclip import contrastive

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.81s/it]
Device set to use cuda:0


In [4]:
os.chdir('../')


In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
os.chdir(os.path.join(os.getcwd(), '../'))

In [4]:
struct = pd.read_pickle('../data/structural/structural_embeddings_0_2000.pkl')
del struct

In [10]:
args = {
    "sequence_embeddings": "../data/esm2/sequence_embedding_t33_0_2000.pkl",
    'structural_embed':'../data/structural/structural_embeddings_0_2000.pkl',
    "functional_embeddings_file": "protclip_embed_dataset.parquet",
    'unitnorm': True,
    'batch_size': 256,
    'dim': 128,
    'nhidden': 1,
    'learning_rate':1e-4,
    'out':'training',
    'name':'testrun',
    'max_epochs': 1000,
    
}

In [11]:
pl.seed_everything(seed=6489)


[rank: 0] Seed set to 6489


6489

In [12]:
structural_embeddings = None
with open(args['structural_embed'], "rb") as f:
    structural_embeddings = pickle.load(f)
structural_embeddings = dict(zip(
    structural_embeddings["protein_id"],
    structural_embeddings["embedding"]
))
list(structural_embeddings.items())[0]

('Q15375',
 array([ -5.27637  ,   9.696514 ,  11.407264 , ...,  -5.5494313,
        -11.50502  , -10.721539 ], dtype=float32))

In [13]:
sequence_embeddings = None
with open(args['sequence_embeddings'], "rb") as f:
    sequence_embeddings = pickle.load(f)

list(sequence_embeddings.items())[0]

('Q15375',
 array([ 0.00049137, -0.04332284, -0.01638226, ..., -0.10839974,
         0.02102051,  0.13916047], dtype=float32))

In [14]:
functional_embeddings = None
df = pd.read_parquet(args["functional_embeddings_file"])
functional_embeddings: dict[str, str] = dict(df[["id", "textual_embedding"]].values)


In [16]:
shared_keys = sorted(set(sequence_embeddings.keys()).intersection(set(structural_embeddings.keys())).intersection(functional_embeddings.keys()))


In [17]:
random.seed(42)  # Set your desired seed here
random.shuffle(shared_keys)


In [18]:
type(functional_embeddings)

dict

In [19]:
from proteinclip.data_utils import CLIPDataset3D
dset = data_utils.CLIPDataset3D(
    triples=shared_keys,
    map1=functional_embeddings,
    map2=sequence_embeddings,
    map3=structural_embeddings,
    enforce_unit_norm=args['unitnorm'],
)

Checking for missing/zero embeddings: 100%|██████████| 1997/1997 [00:00<00:00, 13776.90it/s]
INFO:root:Trimmed 0 triples with missing/zero embeddings; 1997 remain.


In [20]:
split_indices = data_utils.random_split(len(dset), [0.9, 0.05, 0.05])
dset_splits = [data.Subset(dset, idx) for idx in split_indices]

In [21]:
# Create data loaders
train_dl, valid_dl, _test_dl = [
    data.DataLoader(
        ds,
        batch_size=args['batch_size'],
        shuffle=(i == 0),
        # drop_last=(i == 0),
        num_workers=8,
        pin_memory=True,
    )
    for i, ds in enumerate(dset_splits)
]


/home/hice1/savunuri3/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [22]:
# Define network
input_dim_1 = next(iter(train_dl))["x_1"].shape[-1]
input_dim_2 = next(iter(train_dl))["x_2"].shape[-1]
input_dim_3 = next(iter(train_dl))["x_3"].shape[-1]
model_class = (
    contrastive.ContrastiveEmbedding3D
)
net = model_class(
    input_dim_1=input_dim_1,
    input_dim_2=input_dim_2,
    input_dim_3=input_dim_3,
    shared_dim=args['dim'],
    num_hidden=args['nhidden'],
    lr=args['learning_rate'],
)

In [23]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
early_stop_callback = EarlyStopping(
    monitor='val_loss',         # metric name to monitor
    patience=50,                 # epochs with no improvement after which to stop
    mode='min',                 # 'min' if you want to minimize the metric
    verbose=True                # optional: prints a message when triggered
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=1,
    mode='min',
    filename='best-checkpoint'
)


In [24]:
# Train
trainer = pl.Trainer(
    max_epochs=1000,
    accelerator="cuda",
    devices=1,
    enable_progress_bar=True,
    # logger=logger,
    # log_every_n_steps=10,
    deterministic=True,
    callbacks=[early_stop_callback, checkpoint_callback],

)
trainer.fit(net, train_dataloaders=train_dl, val_dataloaders=valid_dl)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/hice1/savunuri3/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float

/home/hice1/savunuri3/.local/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (29) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 29/29 [00:04<00:00,  7.01it/s, v_num=2509879, train_loss=1.190, val_loss=3.220]

Metric val_loss improved. New best score: 3.223


Epoch 1: 100%|██████████| 29/29 [00:00<00:00, 43.26it/s, v_num=2509879, train_loss=0.922, val_loss=2.950]

...

Epoch 412: 100%|██████████| 29/29 [00:00<00:00, 42.04it/s, v_num=2509879, train_loss=0.0087, val_loss=1.940]

Metric val_loss improved by 0.013 >= min_delta = 0.0. New best score: 1.942


Epoch 462: 100%|██████████| 29/29 [00:00<00:00, 38.22it/s, v_num=2509879, train_loss=0.00296, val_loss=2.010]

Monitored metric val_loss did not improve in the last 50 records. Best score: 1.942. Signaling Trainer to stop.


Epoch 462: 100%|██████████| 29/29 [00:00<00:00, 38.13it/s, v_num=2509879, train_loss=0.00296, val_loss=2.010]


In [26]:



# Export model as ONNX files
contrastive.model_to_onnx(
    net.project_1,
    "project3d_1.onnx",
    input_shape=(input_dim_1,),
)
contrastive.model_to_onnx(
    net.project_2,
    "project3d_2.onnx",
    input_shape=(input_dim_2,),
)
contrastive.model_to_onnx(
    net.project_3,
    "project3d_3.onnx",
    input_shape=(input_dim_3,),
)


2025-04-21 22:52:53.612941261 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3066267, index: 36, mask: {37, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-21 22:52:53.616865080 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3066236, index: 5, mask: {6, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-21 22:52:53.618623127 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3066268, index: 37, mask: {38, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-21 22:52:53.620847119 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 3066234, index: 3, mask: {4, }, error code: 22 error msg: Invalid argument. Specify t